# Export anthropogenic heat flux
- This script is used to export anthropogenic heat flux over sites;
- Simulations: CNTL, WRF_LCZ, LI_LCZ, CESM_LCZ;

In [1]:
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
home_path = '/gws/nopw/j04/duicv/yuansun/'
archive = '0_lcz_sp/archive/'
location = 'output_analysis/sp/anthro/'
output_dir = home_path + '0_lcz_sp/output_analysis/sp/anthro/data_for_figure'
psites = ["AU-Preston","AU-SurreyHills","CA-Sunset","FI-Kumpula","FI-Torni",
          "FR-Capitole","GR-HECKOR","JP-Yoyogi","KR-Jungnang","KR-Ochang",
          "MX-Escandon","NL-Amsterdam","PL-Lipowa","PL-Narutowicza","SG-TelokKurau06",
          "UK-KingsCollege","UK-Swindon","US-Baltimore","US-Minneapolis1","US-Minneapolis2",
          "US-WestPhoenix"]
msites = ["AU-Pre","AU-Sur","CA-Sun","FI-Kum","FI-Tor",
          "FR-Cap","GR-HEC","JP-Yoy","KR-Jun","KR-Och",
          "MX-Esc","NL-Ams","PL-Lip","PL-Nar","SG-Tel",
          "UK-Kin","UK-Swi","US-Bal","US-Mi1","US-Mi2",
          "US-Wes"]
START = ['1993', '1994', '2002', '2000', '2000',
        '1994', '2009', '2006', '2007', '2005', 
        '2001', '2009', '1998', '1998', '1996', 
        '2002', '2001', '1992', '1996', '1996', 
        '2001']

start_date = ['2003-08-12T03:30:00', '2004-02-23T05:00:00', '2012-01-01T00:00:00', '2010-12-31T22:30:00', '2010-12-31T22:30:00',
              '2004-02-20T00:30:00', '2019-06-30T22:00:00', '2016-03-31T15:00:00', '2017-01-24T16:00:00', '2015-06-07T15:00:00',
              '2011-06-01T17:00:00', '2019-01-01T00:00:00', '2008-01-01T00:00:00', '2008-01-01T00:00:00', '2006-04-30T16:30:00',
              '2012-04-04T00:00:00', '2011-05-11T19:00:00', '2002-01-01T05:00:00', '2006-06-01T18:00:00', '2006-06-01T18:00:00',
              '2011-12-16T18:30:00']
end_date = ['2004-11-28T11:30:00', '2004-07-19T22:00:00', '2016-12-31T22:00:00', '2013-12-31T20:30:00', '2013-12-31T20:30:00',
            '2005-02-28T22:30:00', '2020-06-30T20:00:00', '2020-03-31T11:00:00', '2019-04-29T05:00:00', '2017-07-26T00:00:00',
            '2012-09-13T12:30:00', '2020-10-13T08:30:00', '2012-12-31T20:00:00', '2012-12-31T20:00:00', '2007-03-31T14:30:00',
            '2013-12-31T21:00:00', '2013-04-25T08:00:00', '2007-01-01T01:00:00', '2009-05-29T11:30:00', '2008-01-01T00:00:00',
            '2013-01-01T05:00:00']
mid_date = ['2004-01-01 00:00:00', '2004-05-01 00:00:00', '2014-01-01 00:00:00', '2012-01-01 00:00:00', '2012-01-01 00:00:00',
            '2004-09-01T00:00:00', '2020-01-01T00:00:00', '2018-01-01T00:00:00', '2018-01-01T00:00:00', '2016-01-01T00:00:00',
            '2012-01-01T00:00:00', '2020-01-01T00:00:00', '2010-01-01T00:00:00', '2010-01-01T00:00:00', '2007-01-01T00:00:00',
            '2013-01-01T00:00:00', '2012-01-01T00:00:00', '2004-01-01T00:00:00', '2008-01-01T00:00:00', '2009-05-29T11:30:00',
            '2012-06-01T00:00:00']

observation = [11, 9.7, 12.8, 6.8, 14.2, 
               47.5, 17.4, 20, 92.7, 3.3, 
               11, 43.4, 7.1, 8.8, 11,
               78.5, 8, 5.5, 1.8, 1.8,
               21.1]
var3 = ['URBAN_AC', 'URBAN_HEAT', 'WASTEHEAT', 'VENTILATION', 'AHF']

index_1 = 7091
index_30 = 365*48
index_60 = 365*24
index_14 = 16077
annual_index = [index_30, index_1, index_30, index_30, index_30,
                index_30, index_30, index_60, index_30, index_30,
                index_30, index_30, index_60, index_60, index_14,
                index_30, index_30, index_60, index_30, index_30,
                index_30]

index = [1,14] is less than 1 year

In [6]:
#for sitesequence in range(len(psites)):
all_results = []
for sitesequence in range(len(psites)):
    GRIDNAME = psites[sitesequence]
    GRIDNAME_single = msites[sitesequence]
    START_year = START[sitesequence]
    start_time = start_date[sitesequence]
    end_time = end_date[sitesequence]
    mid_time = mid_date[sitesequence]
    default = home_path + archive + '0sp/' + GRIDNAME_single + '_def/lnd/hist/' + GRIDNAME_single + '_def.clm2.h0.' + START_year +'-01-01-00000.nc'
    ds_def = xr.open_dataset(default)
    wrf = home_path + archive + '0sp/' + GRIDNAME_single + '_wrf_all_new/lnd/hist/' + GRIDNAME_single + '_wrf_all_new.clm2.h0.' + START_year +'-01-01-00000.nc'
    ds_wrf = xr.open_dataset(wrf)
    li = home_path + archive + '0sp/' + GRIDNAME_single + '_li_all/lnd/hist/' + GRIDNAME_single + '_li_all.clm2.h0.' + START_year +'-01-01-00000.nc'
    ds_li = xr.open_dataset(li)
    cesm = home_path + archive + '0sp/' + GRIDNAME_single + '_cesmlcz/lnd/hist/' + GRIDNAME_single + '_cesmlcz.clm2.h0.' + START_year +'-01-01-00000.nc'
    ds_cesm = xr.open_dataset(cesm)
    
    ds_def['time']=ds_def['time'].dt.round('min')
    ds_def['time']=ds_def['time'].dt.ceil('min')
    ds_wrf['time']=ds_wrf['time'].dt.round('min')
    ds_wrf['time']=ds_wrf['time'].dt.ceil('min')
    ds_li['time']=ds_li['time'].dt.round('min')
    ds_li['time']=ds_li['time'].dt.ceil('min')
    ds_cesm['time']=ds_cesm['time'].dt.round('min')
    ds_cesm['time']=ds_cesm['time'].dt.ceil('min')
    
    ds_def['AHF'] = ds_def['URBAN_HEAT']+ ds_def['WASTEHEAT']
    ds_wrf['AHF'] = ds_wrf['URBAN_HEAT']+ ds_wrf['WASTEHEAT'] 
    ds_li['AHF'] = ds_li['URBAN_HEAT']+ ds_li['WASTEHEAT']
    ds_cesm['AHF'] = ds_cesm['URBAN_HEAT']+ ds_cesm['WASTEHEAT']
    
    sliced_ds_def = ds_def.sel(time=slice(start_time, end_time))[var3]
    df_def = sliced_ds_def.to_dataframe().reset_index()
    sliced_ds_wrf = ds_wrf.sel(time=slice(start_time, end_time))[var3]
    df_wrf = sliced_ds_wrf.to_dataframe().reset_index()
    sliced_ds_li = ds_li.sel(time=slice(start_time, end_time))[var3]
    df_li = sliced_ds_li.to_dataframe().reset_index()
    sliced_ds_cesm = ds_cesm.sel(time=slice(start_time, end_time))[var3]
    df_cesm = sliced_ds_cesm.to_dataframe().reset_index()
    
    annual_df_def = df_def['AHF'].rolling(window=annual_index[sitesequence], min_periods=annual_index[sitesequence]).mean().mean()
    annual_df_wrf = df_wrf['AHF'].rolling(window=annual_index[sitesequence], min_periods=annual_index[sitesequence]).mean().mean()
    annual_df_li = df_li['AHF'].rolling(window=annual_index[sitesequence], min_periods=annual_index[sitesequence]).mean().mean()
    annual_df_cesm = df_cesm['AHF'].rolling(window=annual_index[sitesequence], min_periods=annual_index[sitesequence]).mean().mean()
    
    results = []
    results.append({
        'site': GRIDNAME,
        'obs': observation[sitesequence],
        'def': annual_df_def,
        'wrf': annual_df_wrf,
        'li': annual_df_li,
        'cesm': annual_df_cesm
    })
    all_results.extend(results) 
    
results_df = pd.DataFrame(all_results)
results_df.to_csv(output_dir + 'ahf.csv', index = False)
results_df           

,site,obs,def,wrf,li,cesm
0,AU-Preston,11.0,3.673763,0.357257,0.379587,2.227173
1,AU-SurreyHills,9.7,4.702309,0.442259,0.462739,2.931951
2,CA-Sunset,12.8,2.691127,1.025522,1.030529,2.300033
3,FI-Kumpula,6.8,7.029572,5.433534,5.422522,8.027582
4,FI-Torni,14.2,22.683951,16.969729,16.100199,29.292945
5,FR-Capitole,47.5,6.255516,4.804058,4.715139,10.347942
6,GR-HECKOR,17.4,0.092012,0.034314,0.027888,0.624911
7,JP-Yoyogi,20.0,1.806313,1.089640,1.071925,3.015803
8,KR-Jungnang,92.7,6.959473,6.971582,6.927211,10.547965
9,KR-Ochang,3.3,2.596283,2.243995,2.173448,3.372545
